In [181]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit

In [182]:
FILE_PATH = os.path.join(os.getcwd(), 'data')
MUSIC_DURATION = 30

In [183]:
class GenreNode:
    def __init__(self, name):
        self.name = name
        self.music_files = []

    def add_music_file(self, music_file):
        self.music_files.append(music_file)

class MusicGenreTree:
    def __init__(self):
        self.genres = {}

    def add_genre_and_music(self, genre_name, music_file):
        if genre_name not in self.genres:
            self.genres[genre_name] = GenreNode(genre_name)
        self.genres[genre_name].add_music_file(music_file)

    def get_genre_music_dict(self):
        genre_music_dict = {genre: node.music_files for genre, node in self.genres.items()}
        return genre_music_dict

def build_music_genre_tree_from_directory(root_directory):
    tree = MusicGenreTree()
    for root, dirs, files in os.walk(root_directory):
        genre_name = os.path.basename(root)
        for file in files:
            if file.endswith(".mp3") or file.endswith(".wav"):
                tree.add_genre_and_music(genre_name, os.path.join(root, file))
    return tree

def extract_mfcc_features(file_path, n_mfcc=13):
    try:
        y, sr = librosa.load(file_path, duration=60)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        
        mfccs_mean = np.mean(mfccs, axis=1)
        mfccs_std = np.std(mfccs, axis=1)
        
        features = np.concatenate((mfccs_mean, mfccs_std))
        return features
    except Exception as e:
        print(file_path)
        print(f"Error processing {file_path}: {e}")
        return None

def get_feature_matrix_and_labels(genre_music_dict):
    feature_list = []
    label_list = []

    for genre, music_files in genre_music_dict.items():
        for music_file in music_files:
            features = extract_mfcc_features(music_file)
            if features is not None:
                feature_list.append(features)
                label_list.append(genre)

    feature_matrix = np.array(feature_list)
    label_vector = np.array(label_list)

    return feature_matrix, label_vector

In [184]:
tree = build_music_genre_tree_from_directory(FILE_PATH)
genre_music_dict = tree.get_genre_music_dict()

In [185]:
feature_matrix, label_vector = get_feature_matrix_and_labels(genre_music_dict)

In [186]:
std_scalar = StandardScaler()
X = std_scalar.fit_transform(feature_matrix)

In [187]:
encoder = LabelEncoder()
y = encoder.fit_transform(label_vector)

In [188]:
stratified_splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, test_index in stratified_splitter.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

print(f"Train set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")

Train set size: 799
Test set size: 200


In [189]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.optimizers import Adam

In [190]:
y_train_categorical = tf.keras.utils.to_categorical(y_train, 10)
y_test_categorical = tf.keras.utils.to_categorical(y_test, 10)

In [178]:
model = Sequential()

# Add layers to the model
model.add(Input(shape=(26,)))
model.add(Dense(128, activation='relu'))  # Increase neurons
model.add(Dropout(0.5))  # Regularization
model.add(Dense(64, activation='relu'))  # Additional hidden layer
model.add(Dense(32, activation='relu'))  # Additional hidden layer
model.add(Dense(10, activation='softmax'))  # Output layer

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print summary of the model architecture
model.summary()

Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_116 (Dense)                    │ (None, 128)                 │           3,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_26 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_117 (Dense)                    │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_118 (Dense)                    │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_119 (Dense)                    │ (None, 10)                  │             330 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,122 (55.16 KB)

 Trainable params: 14,122 (55.16 KB)

 Non-trainable params: 0 (0.00 B)

In [179]:
history = model.fit(X_train, y_train_categorical, epochs=100, validation_data=(X_test, y_test_categorical), verbose=1)

Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.1386 - loss: 2.2959 - val_accuracy: 0.3050 - val_loss: 2.1469
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3138 - loss: 2.0634 - val_accuracy: 0.3700 - val_loss: 1.9357
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3570 - loss: 1.8559 - val_accuracy: 0.4200 - val_loss: 1.6942
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4367 - loss: 1.6448 - val_accuracy: 0.4900 - val_loss: 1.5004
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4572 - loss: 1.4847 - val_accuracy: 0.5100 - val_loss: 1.4008
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5213 - loss: 1.3386 - val_accuracy: 0.5300 - val_loss: 1.3272
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5765 - loss: 1.2015 - val_accuracy: 0.5500 - val_loss: 1.2604
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5442 - loss: 1.2158 - val_accuracy: 0.5300 - 

In [180]:
loss, accuracy = model.evaluate(X_test, y_test_categorical)
print(f"Test Accuracy: {accuracy:.2f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7057 - loss: 1.2136 
Test Accuracy: 0.71
